In [1]:
import numpy as np
import pandas as pd
import pickle

username='ozaltun'
def save_obj(obj, loc ):
    with open(loc, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(loc):
    with open(loc, 'rb') as f:
        return pickle.load(f)

def reverse_dict(my_map):
    inv_map = {v: k for k, v in my_map.items()}
    return inv_map

reg_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/reg_2_num.pickle')

comm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/comm_2_num.pickle')

prodcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/prodcomm_2_num.pickle')
nsavcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/nsavcomm_2_num.pickle')
demdcomm_2_num = load_obj('/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/mapping/demdcomm_2_num.pickle')


In [2]:
root = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/GTAP/matrices/{}.pickle'



In [3]:
F_goods = ['ofd','b_t','tex','wap','lea','lum','ppp','p_c','crp','nmm','fmp','mvh','otn','ele',\
           'ome','omf','ely','gdt','wtr','cns','trd','otp','wtp','atp','cmn','ofi','isr','obs','ros','osg']

C_goods = ['pdr', 'wht','gro','v_f','osd','c_b','pfb','ocr','ctl','oap','rmk','wol','frs','fsh','coa','oil',\
           'gas','omn','cmt','omt','vol','mil','pcr','sgr','i_s','nfm']

In [4]:
C_goods[21]

'mil'

In [5]:
F_goods[0]

'ofd'

In [6]:
data = {}
n = 134
g = len(C_goods)
k = len(F_goods)

data['n'] = n
data['g'] = g
data['k'] = k

In [7]:
# Lets get the easy one out of the way
data['E'] = np.around(load_obj(root.format('E')).reshape((n, 1)), decimals=6)

In [8]:
# Now production:
Y = np.around(load_obj(root.format('Y_v2')), decimals=6)
data['Y_g'] = np.zeros((n, g))
data['Y_k'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['Y_g'][:, i] = Y[comm_2_num[key], :]


for i, key in enumerate(F_goods):
    data['Y_k'][:, i] = Y[comm_2_num[key], :]

In [9]:
# Now demand:
D = np.around(load_obj(root.format('D')), decimals=6)
data['D_g_old'] = np.zeros((n, g))
data['D_k_old'] = np.zeros((n, k))


for i, key in enumerate(C_goods):
    data['D_g_old'][:, i] = D[comm_2_num[key], :]


for i, key in enumerate(F_goods):
    data['D_k_old'][:, i] = D[comm_2_num[key], :]

In [10]:
# Now trade
X = np.around(load_obj(root.format('X_v2')), decimals=6)

data['X_g'] = np.zeros((n, n, g))
data['X_k'] = np.zeros((n, n, k))


for i, key in enumerate(C_goods):
    data['X_g'][:, :, i] = X[comm_2_num[key], :, :].T

for i, key in enumerate(F_goods):
    data['X_k'][:, :, i] = X[comm_2_num[key], :, :].T

In [11]:
# Adjust local consumption
for i in range(n):
    temp = np.zeros(g)
    for j in range(n):
        if i != j:
            temp += data['X_g'][j, i, :]
        
    data['X_g'][i, i, :] = data['Y_g'][i,:] - temp

# Re-adjust with demand figures
idx_g = np.where(data['X_g'] < 0)
print(idx_g)

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [12]:
# data['X_g'][idx_g] = data['D_g_old'][idx_g[1:]]

# # Now go back 
# for i in range(idx_g[0].shape[0]):
#     print(idx_g[1][i], idx_g[2][i])
#     data['Y_g'][idx_g[1][i], idx_g[2][i]] += data['X_g'][:, idx_g[1][i], idx_g[2][i]].sum()

In [13]:
#data['X'][data['X']<0]

In [14]:
for i in range(n):
    temp = np.zeros(k)
    for j in range(n):
        if i != j:
            temp += data['X_k'][j, i, :]
        
    data['X_k'][i, i, :] = data['Y_k'][i,:] - temp

idx_k = np.where(data['X_k'] < 0)
print(idx_k)

# data['X_k'][idx_k] = data['D_k_old'][idx_k[1:]]

 
# for i in range(idx_k[0].shape[0]):
#     print(idx_k[1][i], idx_k[2][i])
#     data['Y_g'][idx_k[1][i], idx_k[2][i]] += data['X_k'][:, idx_k[1][i], idx_k[2][i]].sum()

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))


In [15]:
data['D_g'] = data['X_g'].sum(axis=1)
data['D_k'] = data['X_k'].sum(axis=1)

In [16]:
## Supply == Demand

print(data['Y_k'].sum(axis=1).sum(axis=0) - data['D_k'].sum(axis=0).sum())
print(data['Y_g'].sum(axis=1).sum(axis=0) - data['D_g'].sum(axis=0).sum())

4.470348358154297e-08
0.0


In [17]:
# Nice !
print((data['X_k']/data['Y_k'].reshape((1, n, k))).sum(axis=0))
print((data['X_g']/data['Y_g'].reshape((1, n, g))).sum(axis=0))



[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]
[[1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 ...
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]
 [1. 1. 1. ... 1. 1. 1.]]


In [18]:
# Trade fraction
data['lambda_g'] = data['X_g']/data['D_g'].reshape((n, 1, g))
data['lambda_k'] = data['X_k']/data['D_k'].reshape((n, 1, k))

In [19]:
data['lambda_k'] = data['lambda_k'] /data['lambda_k'].sum(axis=1).reshape((n, 1, k))

In [20]:
data['lambda_g'] = data['lambda_g'] /data['lambda_g'].sum(axis=1).reshape((n, 1, g))

In [21]:
# Now interlinkage
d = np.around(load_obj(root.format('d_l')), decimals=6)

data['d'] = np.zeros((n, k, g))

for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['d'][:, j, i] = d[comm_2_num[comm], comm_2_num[final], :]

In [22]:
np.where(data['d'].sum(axis=1) == 0)

(array([], dtype=int64), array([], dtype=int64))

In [23]:
k

30

In [24]:
data['d'] = data['d']/data['d'].sum(axis=1).reshape((n, 1, g))
data['d'].sum(axis=1)

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [25]:
# Share of labor
e_labor = np.around(load_obj(root.format('e_labor')), decimals=6)

data['e_L'] = e_labor.reshape((n, 1))


# Share of commodities
e_comm = load_obj(root.format('e_comm'))
data['e'] = np.zeros((n, g))

for i, key in enumerate(C_goods):
    data['e'][:, i] = e_comm[comm_2_num[key], :]


In [26]:
data['e_L'] =( 1 - data['e'].sum(axis=1)).reshape((n, 1))

In [27]:
# alpha
alpha = np.around(load_obj(root.format('alpha')), decimals=6)
data['alpha'] = np.zeros((n, k))

for i, key in enumerate(F_goods):
    data['alpha'][:, i] = alpha[comm_2_num[key], :]

In [28]:
data['alpha'].shape

(134, 30)

In [29]:
# Doing this so that equation (15) works out
data['alpha'] = data['alpha']/data['alpha'].sum(axis=1).reshape(134, 1)

In [30]:
idx_k

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [31]:
idx_g

(array([], dtype=int64), array([], dtype=int64), array([], dtype=int64))

In [32]:
# phi
phi_labor = np.around(load_obj(root.format('phi_labor')), decimals=6)
phi_comm = np.around(load_obj(root.format('phi_comm')), decimals=6)
phi_natlres = np.around(load_obj(root.format('phi_natlres')), decimals=6) # See if you need to integrate land
phi_land = np.around(load_obj(root.format('phi_land')), decimals=6)

data['phi_L_g'] = np.zeros((n, g))
data['phi_L_k'] = np.zeros((n, k))

data['phi_R'] = np.zeros((n, g))

data['phi'] = np.zeros((n, k, g))

for i, key in enumerate(C_goods):
    data['phi_L_g'][:, i] = phi_labor[comm_2_num[key], :]
    data['phi_R'][:, i] = phi_natlres[comm_2_num[key], :] + phi_land[comm_2_num[key], :]
    
for i, key in enumerate(F_goods):    
    data['phi_L_k'][:, i] = phi_labor[comm_2_num[key], :]
    
for i, comm in enumerate(C_goods):
    for j, final in enumerate(F_goods):
        data['phi'][:, j, i] = phi_comm[comm_2_num[comm], comm_2_num[final], :]

In [33]:
np.where(data['phi'].sum(axis=2) == 0)

(array([  3,   5,   5,   5,   7,   8,   8,   8,  13,  13,  13,  13,  16,
         17,  20,  20,  21,  22,  27,  29,  29,  30,  30,  30,  33,  33,
         34,  34,  35,  35,  35,  37,  37,  37,  38,  40,  40,  41,  41,
         41,  43,  43,  43,  49,  50,  58,  62,  62,  64,  64,  67,  67,
         79,  79,  85,  89,  89,  89, 105, 107, 107, 109, 109, 109, 113,
        113, 117, 117, 117, 117, 117, 121, 124, 124, 126, 126, 129, 129,
        131, 131, 131, 133]),
 array([25, 23, 25, 26, 12, 24, 25, 26, 22, 23, 24, 26, 26, 26, 24, 25, 23,
        26, 26, 25, 26, 24, 25, 26, 22, 23, 22, 23, 24, 25, 26, 22, 23, 26,
        23, 25, 26, 21, 22, 23, 16, 25, 26, 26, 25, 26, 25, 26, 22, 26, 25,
        26, 25, 26, 26, 24, 25, 26,  9, 25, 26, 15, 24, 25, 24, 25, 22, 23,
        24, 25, 26, 23, 24, 25, 25, 26, 23, 24, 24, 25, 26, 26]))

In [42]:
np.where(data['phi'].sum(axis=2) == 1)

(array([], dtype=int64), array([], dtype=int64))

In [43]:
data['phi_L_k'] = 1 - data['phi'].sum(axis=2)

In [44]:
# normalizer = (data['phi_L_k'] + data['phi'].sum(axis=2))
# data['phi'] = data['phi']/(normalizer.reshape(n, k, 1))
# data['phi_L_k'] = data['phi_L_k']/normalizer

In [45]:
(data['phi_L_k'] + data['phi'].sum(axis=2))

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [46]:
# Update based on notes:
# data['phi'] = data['d']*data['D_g'].reshape((n, 1, g))/data['Y_k'].reshape((n, k, 1))

# data['phi_L_k'] = 1 - data['phi'].sum(axis=2)

#print(np.where(data['phi'] >1),'blah', np.where(data['phi_L_k'] <=0))


In [47]:
data['phi_L_g']

array([[0.442945, 0.190069, 0.26946 , ..., 0.151499, 0.173513, 0.087179],
       [0.180395, 0.258239, 0.268929, ..., 0.203775, 0.187162, 0.167262],
       [0.233072, 0.16678 , 0.196893, ..., 0.172642, 0.190792, 0.164554],
       ...,
       [0.162401, 0.096259, 0.140955, ..., 0.043654, 0.126985, 0.091982],
       [0.059207, 0.113961, 0.215515, ..., 0.043239, 0.139094, 0.125107],
       [0.237364, 0.281232, 0.258127, ..., 0.189792, 0.094032, 0.155634]])

In [48]:
import matplotlib.pyplot as plt
data['phi_L_g'] = 1 - data['phi_R']
data['phi_L_g'][data['phi_L_g']<0]

array([], dtype=float64)

In [49]:
data['theta_k'] = np.ones((k, 1))*5.
data['theta_g'] = np.ones((g, 1))*5.

data['eta'] = np.ones((k, 1))*0.4
data['rho_g'] = np.ones((g, 1))*0.4

data['sigma'] = 0.4

In [50]:
### Change non zero elements to zero ###

In [51]:
for i in data.keys():
    print(i)
    if i not in ['n', 'g','k','sigma']:
        data[i] = np.around(data[i], decimals=4)
        data[i][data[i]<0] = 0

n
g
k
E
Y_g
Y_k
D_g_old
D_k_old
X_g
X_k
D_g
D_k
lambda_g
lambda_k
d
e_L
e
alpha
phi_L_g
phi_L_k
phi_R
phi
theta_k
theta_g
eta
rho_g
sigma


In [52]:
data['E_L'] = (data['phi_L_g']*data['Y_g']).sum(axis=1) + (data['phi_L_k']*data['Y_k']).sum(axis=1)

In [53]:
loc = '/Users/'+username+'/Dropbox (MIT)/Data/Trade/general_equilibrium_gravity/final_data/data_20200226.pickle'
save_obj(data, loc)